# QAAS: Quantum Agent Annealed Swarm

1000 intents, 48 agents, token-based cost model, CQM with dependency chains. Simulated annealing vs greedy.

Cost = estimated_tokens x token_rate per agent. Story points (Fibonacci: 1-2-3-5-8-13) are reporting-only — the CQM stays token-based. Sprint economics track cost/SP and capacity projections.

Code is organized into modules: `config.py`, `agents.py`, `intents.py`, `model.py`, `solve.py`, `report.py`. This notebook is a thin orchestrator.

## Setup

Install D-Wave Ocean SDK:
```bash
pip install dwave-ocean-sdk
```

Configure your API token:
```bash
dwave config create
```

In [ ]:
from agents import build_agent_pool
from config import STORY_POINTS
from intents import generate_intents, build_workflow_chains, DISTRIBUTION
from model import build_cqm
from solve import solve_sa, solve_hybrid, parse_assignments, greedy_solve
from report import print_shift_report, print_comparison

## Define the Swarm

48 agents across 4 cloud model tiers and 8 local models.
Cloud agents have 25 task capacity each, local agents are limited but free.

In [ ]:
agents, agent_names = build_agent_pool()

num_agents = len(agent_names)
cloud_cap = sum(a['capacity'] for a in agents.values() if not a['is_local'])
local_cap = sum(a['capacity'] for a in agents.values() if a['is_local'])

print(f"Total agents: {num_agents}")
print(f"Cloud: {sum(1 for a in agents.values() if not a['is_local'])} agents, {cloud_cap} capacity")
print(f"Local: {sum(1 for a in agents.values() if a['is_local'])} agents, {local_cap} capacity")
print(f"Total capacity: {cloud_cap + local_cap}")

## Define the Intents

1000 tasks across 6 complexity tiers mapped to Fibonacci story points (1-2-3-5-8-13).
3250 total SP. Workflow dependency chains model real feature-dev, bug-fix, and infra flows.

In [ ]:
intents = generate_intents()
workflow_chains = build_workflow_chains(intents)

num_intents = len(intents)
num_chained = sum(1 for t in intents if t['depends'])

print(f"Total tasks: {num_intents}")
print(f"Workflow chains: {len(workflow_chains)} ({num_chained} dependent tasks)")
print(f"  feature-dev: {sum(1 for t, _ in workflow_chains if t == 'feature-dev')}")
print(f"  bug-fix:     {sum(1 for t, _ in workflow_chains if t == 'bug-fix')}")
print(f"  infra:       {sum(1 for t, _ in workflow_chains if t == 'infra')}")

total_sp = sum(t['story_points'] for t in intents)
print(f"\nTotal story points: {total_sp}")
print(f"\n{'Tier':<15} {'Count':>5} {'SP':>4} {'Total SP':>9} {'Min Quality':>12}")
print(f"{'─' * 48}")
for c, count, min_q in DISTRIBUTION:
    sp = STORY_POINTS[c]
    tier_count = sum(1 for t in intents if t['complexity'] == c)
    print(f"{c:<15} {tier_count:>5} {sp:>4} {tier_count * sp:>9} {min_q:>12.2f}")

## Build the CQM

CQM separates objective from constraints. Assignment and capacity are hard constraints;
dependency quality ordering is a soft penalty in the objective.

In [ ]:
cqm, x = build_cqm(intents, agents, agent_names)

## Solve with Simulated Annealing

Convert CQM to BQM with automatic slack variables, then run SA locally.

In [ ]:
sampleset = solve_sa(cqm)
assignments = parse_assignments(sampleset, agent_names)

In [ ]:
print_shift_report(assignments, intents, agents, workflow_chains)

## Solve with D-Wave (Quantum)

CQM goes directly to the hybrid solver — no manual BQM conversion needed.
Requires a D-Wave Leap API token.

In [ ]:
# Uncomment to run on real quantum hardware

# sampleset = solve_hybrid(cqm)
# feasible = sampleset.filter(lambda s: s.is_feasible)
# if feasible:
#     assignments = parse_assignments(feasible, agent_names)
#     print_shift_report(assignments, intents, agents, workflow_chains)

## Greedy vs Annealing

Greedy picks the cheapest valid agent per task, first-come first-served. No global optimization, no dependency awareness.

In [ ]:
greedy_assignments, greedy_cost = greedy_solve(intents, agents)
print_comparison(assignments, greedy_assignments, greedy_cost, intents, agents)